In [1]:
import numpy as np 
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

J:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Determine sizes of train and test data
print("train rows =", train.shape[0])
print("test rows =", test.shape[0])

train rows = 19579
test rows = 8392


In [3]:
def stemmText(row):
    source = row["text"]
    
    text_string = "".join(l for l in source if l not in string.punctuation)
    
    stemmer = SnowballStemmer("english")       
    words = text_string.split(" ")
        
    new_words = []
        
    for i in range(len(words)):
        if words[i] != "" and words[i] != " ":
            new_words.append(stemmer.stem(words[i].strip()))

    text_string = " ".join(l for l in new_words)
    
    return text_string
        

train['textStemm'] = train.apply(stemmText, axis=1, raw=False)
test['textStemm'] = test.apply(stemmText, axis=1, raw=False)

In [4]:
train

,id,text,author,textStemm
0,id26305,"This process, however, afforded me no means of...",EAP,this process howev afford me no mean of ascert...
1,id17569,It never once occurred to me that the fumbling...,HPL,it never onc occur to me that the fumbl might ...
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,how love is spring as we look from windsor ter...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,find noth els not even gold the superintend ab...
5,id22965,"A youth passed in solitude, my best years spen...",MWS,a youth pass in solitud my best year spent und...
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP,the astronom perhap at this point took refug i...
7,id13515,The surcingle hung in ribands from my body.,EAP,the surcingl hung in riband from my bodi
8,id19322,I knew that you could not say to yourself 'ste...,EAP,i knew that you could not say to yourself ster...
9,id00912,I confess that neither the structure of langua...,MWS,i confess that neither the structur of languag...


In [5]:
def convertAuthorLabel(row):
    author = row['author']
    if author == 'EAP':
        return 0
    elif author == 'HPL':
        return 1
    elif author == 'MWS':
        return 2

train['label'] = train.apply(convertAuthorLabel, axis=1, raw=False)
train

,id,text,author,textStemm,label
0,id26305,"This process, however, afforded me no means of...",EAP,this process howev afford me no mean of ascert...,0
1,id17569,It never once occurred to me that the fumbling...,HPL,it never onc occur to me that the fumbl might ...,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,how love is spring as we look from windsor ter...,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,find noth els not even gold the superintend ab...,1
5,id22965,"A youth passed in solitude, my best years spen...",MWS,a youth pass in solitud my best year spent und...,2
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP,the astronom perhap at this point took refug i...,0
7,id13515,The surcingle hung in ribands from my body.,EAP,the surcingl hung in riband from my bodi,0
8,id19322,I knew that you could not say to yourself 'ste...,EAP,i knew that you could not say to yourself ster...,0
9,id00912,I confess that neither the structure of langua...,MWS,i confess that neither the structur of languag...,2


In [6]:
print(train['textStemm'][:10])

0    this process howev afford me no mean of ascert...
1    it never onc occur to me that the fumbl might ...
2    in his left hand was a gold snuff box from whi...
3    how love is spring as we look from windsor ter...
4    find noth els not even gold the superintend ab...
5    a youth pass in solitud my best year spent und...
6    the astronom perhap at this point took refug i...
7             the surcingl hung in riband from my bodi
8    i knew that you could not say to yourself ster...
9    i confess that neither the structur of languag...
Name: textStemm, dtype: object


In [7]:
features_train, features_test, labels_train, labels_test = train_test_split(train['textStemm'], train['label'], test_size=0.2, random_state=42)

In [8]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.999, stop_words='english')   
features_train_transformed = vectorizer.fit_transform(features_train)
print(np.shape(features_train_transformed))

print(labels_train[:5])
train_label_one_hot = to_categorical(labels_train)
print(train_label_one_hot[:5])

(15663, 13949)
5107     0
5465     2
15443    2
9737     2
10192    1
Name: label, dtype: int64
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [9]:
print(len(vectorizer.get_feature_names()))

13949


In [10]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(vectorizer.get_feature_names())))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(features_train_transformed, train_label_one_hot, epochs=100, batch_size=168)

Epoch 1/100
15663/15663 [==============================] - 4s 274us/step - loss: 0.9753 - acc: 0.5796
Epoch 2/100
15663/15663 [==============================] - 4s 255us/step - loss: 0.6808 - acc: 0.8231
Epoch 3/100
15663/15663 [==============================] - 4s 249us/step - loss: 0.4750 - acc: 0.8644
Epoch 4/100
15663/15663 [==============================] - 4s 248us/step - loss: 0.3608 - acc: 0.8838
Epoch 5/100
15663/15663 [==============================] - 4s 250us/step - loss: 0.2944 - acc: 0.9017
Epoch 6/100
15663/15663 [==============================] - 4s 252us/step - loss: 0.2535 - acc: 0.9114
Epoch 7/100
15663/15663 [==============================] - 4s 256us/step - loss: 0.2238 - acc: 0.9198
Epoch 8/100
15663/15663 [==============================] - 5s 306us/step - loss: 0.2018 - acc: 0.9282
Epoch 9/100
15663/15663 [==============================] - 5s 289us/step - loss: 0.1851 - acc: 0.9330
Epoch 10/100
15663/15663 [==============================] - 5s 304us/step - loss: 

15663/15663 [==============================] - 4s 249us/step - loss: 0.0091 - acc: 0.9969
Epoch 81/100
15663/15663 [==============================] - 4s 286us/step - loss: 0.0087 - acc: 0.9971
Epoch 82/100
15663/15663 [==============================] - 4s 247us/step - loss: 0.0084 - acc: 0.9971
Epoch 83/100
15663/15663 [==============================] - 4s 261us/step - loss: 0.0082 - acc: 0.9971 0s - loss: 0.0076 - acc
Epoch 84/100
15663/15663 [==============================] - 4s 248us/step - loss: 0.0078 - acc: 0.9973
Epoch 85/100
15663/15663 [==============================] - 4s 252us/step - loss: 0.0074 - acc: 0.9974
Epoch 86/100
15663/15663 [==============================] - 4s 251us/step - loss: 0.0071 - acc: 0.9973
Epoch 87/100
15663/15663 [==============================] - 4s 248us/step - loss: 0.0071 - acc: 0.9974
Epoch 88/100
15663/15663 [==============================] - 4s 252us/step - loss: 0.0067 - acc: 0.9974
Epoch 89/100
15663/15663 [==============================] - 4s

## Evaluate

### Seperated training data as test data

In [12]:
features_test_transformed  = vectorizer.transform(features_test)
print(labels_test[:5])
test_label_one_hot = to_categorical(labels_test)
print(test_label_one_hot[:5])

score = model.evaluate(features_test_transformed, test_label_one_hot, batch_size=128)
print(score)

6148     0
4881     2
16114    2
738      0
13435    1
Name: label, dtype: int64
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
3916/3916 [==============================] - 1s 130us/step
[1.7328917469992944, 0.7668539328886856]


In [13]:
y_predict = model.predict(features_test_transformed)
print(y_predict)

y_prdict_labels = []
for i in range(0 ,len(y_predict)):
    y_prdict_labels.append(np.argmax(y_predict[i]))

acc2=float((y_prdict_labels==labels_test).sum())/float(len(labels_test))
print("accuracy: {0:.3f}%".format(acc2))

[[9.99999881e-01 9.15353516e-18 6.57565877e-08]
 [2.01095419e-04 1.09207747e-03 9.98706818e-01]
 [1.16942538e-04 4.74525264e-09 9.99883056e-01]
 ...
 [2.55300052e-11 1.00000000e+00 9.30854447e-14]
 [2.63680045e-06 9.99997139e-01 1.87894571e-07]
 [9.50223655e-02 9.04977679e-01 1.30871225e-08]]
accuracy: 0.767%


### Use real testing data

In [14]:
features_real_test_transformed = vectorizer.transform(test['textStemm'])
print(np.shape(features_test_transformed))

(3916, 13949)


In [15]:
realtest_predict = model.predict(features_real_test_transformed)
print(realtest_predict)

realtest_prdict_labels = []
for i in range(0 ,len(realtest_predict)):
    realtest_prdict_labels.append(np.argmax(realtest_predict[i]))
    
print(realtest_prdict_labels)

[[6.3380827e-07 2.2139986e-13 9.9999940e-01]
 [9.9999988e-01 3.0960365e-08 9.4408520e-08]
 [3.5161100e-04 9.9964833e-01 4.4753116e-12]
 ...
 [1.0252753e-06 5.8171872e-16 9.9999893e-01]
 [9.8484230e-01 2.1379788e-12 1.5157711e-02]
 [1.9630458e-12 1.0000000e+00 5.9100064e-20]]
[2, 0, 1, 0, 0, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 2, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 0, 0, 2, 1, 0, 2, 2, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 1, 0, 2, 0, 1, 2, 0, 2, 1, 1, 0, 0, 0, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 2, 2, 1, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 2, 0, 0, 2, 1, 0, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 1, 0, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 2, 2, 1, 1, 0, 2, 0, 0, 0, 2, 0, 2, 1, 0, 